# Import related pacakages

In [2]:
import cobra
import sys
sys.path.append(r'./code/')
from cobrapy_ec_model_function import *

# Inputing files

In [2]:
# The genome-scale metabolic model for constructing the enzyme-constrained model
model_name = './data/iML1515.xml' 

# Reaction-kcat file.
# eg. AADDGT,49389.2889
reaction_kcat_file = "./data/reaction_kcat.csv"

# Gene-abundance file. 
# eg. b0789,1.1
gene_abundance_file = "./data/gene_abundance.csv"

# Gene-molecular_weight file. 
# eg. b0001,thrL,2.13846
gene_molecular_weight_file = "./data/gene_molecular_weight.csv"


# Step1: Preprocessing of model

The reversible reactions in the GEM model are divided into two irreversible reactions. The input is iML1515 with 2712 reactions. The output is a model with 3375 irreversible reactions.

In [3]:
model = cobra.io.read_sbml_model(model_name)
convert_to_irreversible(model)
model

Name,iML1515
Memory address,0x01b644e2ee10
Number of metabolites,1877
Number of reactions,3375
Number of groups,38
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


# Step2: Retrieving enzyme kinetics and proteomics data

The inputs are GEM model. The outputs are 'genes' and 'gpr_relationship' data in the iML1515.

In [4]:
[genes,gpr_relationship] = get_genes_and_gpr(model)

Get the molecular weight of the enzyme (MW) according to the file of all_reaction_GPR.csv, which obtained from the previous step (gpr_relationship, ./analysis/all_reaction_GPR.csv). We need to manually correct the error of the gene_reaction_rule of a small amount of reactions in iML1515 (See Supplementary Table S1 for details), and also need to manually get the subunit of each protein from EcoCyc.

In [5]:
# reaction-gene-subunit-MW file. 
# eg. ALATA_D2,D-alanine transaminase,b2551 or b0870,45.31659 or 36.49471 ,2 or 4 
reaction_gene_subunit_MW = "./data/reaction_gene_subunit_MW.csv"
reaction_mw = calculate_reaction_mw(reaction_gene_subunit_MW)

Calculate kcat/MW. The inputs are 'reaction_kcat' and 'reaction_MW' data for calculating the kcat/MW (When the reaction is catalyzed by several isozymes, the maximum is retained).

In [6]:
reaction_kcat_mw = calculate_reaction_kcat_mw(reaction_kcat_file, reaction_mw)

Calculate f. The input is 'genes' data, 'gene_abundance.csv' and 'gene_molecular_weight.csv'.

In [7]:
f = calculate_f(genes, gene_abundance_file, gene_molecular_weight_file)
f

0.4059986079578236

# Step3: Save enzyme concentration constraint model as json file.

In [8]:
model_file = './data/iML1515.xml' 
reaction_kcat_file="./analysis/reaction_kcat_mw.csv"
#The enzyme mass fraction 
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
sigma = 0.5 
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model(model_file, reaction_kcat_file, f, ptot, sigma , lowerbound, upperbound)

# Step4: Solveing enzyme concentration constraint by COBRApy.

In [4]:
json_model_path="./model/iML1515_irr_enz_constraint.json"
enz_model=get_enzyme_constraint_model(json_model_path)
pfba_solution = cobra.flux_analysis.pfba(enz_model)
pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']

0.7074061852934291

In [5]:
norm_model=cobra.io.json.load_json_model(json_model_path)
pfba_solution = cobra.flux_analysis.pfba(norm_model)
pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']

0.8769972144269698